# Load MNIST

The following code loads MNIST and normalizes it and flattens it.

In [ ]:
from IPython.display import display
from keras.datasets import mnist
from keras.preprocessing.image import array_to_img
import numpy as np

[X_train, _], [X_test, _] = mnist.load_data()
X_train = np.array([x.flatten() for x in X_train.astype('float32')/255.])
X_test = np.array([x.flatten() for x in X_test.astype('float32')/255.])

for x in X_train[:5]:
    img = array_to_img(x.reshape(28, 28, 1))
    display(img)

# Define Sparse Encoder

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.regularizers import l1

ENCODING_DIM = 32

encoder = Sequential(name='encoder')
encoder.add(Dense(units=ENCODING_DIM, activity_regularizer=l1(1e-5), input_shape=[784]))
encoder.add(Activation('sigmoid'))

encoder.summary()
SVG(model_to_dot(encoder).create(prog='dot', format='svg'))

# Define Deep Decoder

In [ ]:
decoder = Sequential(name='decoder')

decoder.add(Dense(units=64, activation='relu', input_shape=[ENCODING_DIM]))
decoder.add(Dense(units=128, activation='relu'))
decoder.add(Dense(units=784, activation='sigmoid'))

decoder.summary()
SVG(model_to_dot(decoder).create(prog='dot', format='svg'))

# Define Autoencoder

In [ ]:
from keras.models import Model
from keras.layers import Input

img = Input(shape=[784])
hidden = encoder(img)
original = decoder(hidden)

autoencoder = Model(inputs=img, outputs=original, name='autoencoder')
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.summary()
SVG(model_to_dot(autoencoder).create(prog='dot', format='svg'))

# Train the Autoencoder

In [ ]:
autoencoder.fit(X_train, X_train, epochs=1, validation_data=(X_test, X_test))

# Reconstruct Test Images

In [ ]:
X_encoded = encoder.predict(X_test)
X_reconstructed = decoder.predict(X_encoded)

print(X_encoded.shape); print(X_reconstructed.shape)

# Visualize Reconstructed Images

In [ ]:
test_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_test]
reconstructed_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_reconstructed]

idxs = np.random.choice(len(X_test), size=5)
for idx in idxs:
    real_img, reconstructed_img = test_imgs[idx], reconstructed_imgs[idx]
    display(real_img); display(reconstructed_img)

# Image Denoising

In [ ]:
noise_factor = 0.5

X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape) 
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape) 
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

for x in X_test_noisy[:5]:
    x_ = x.reshape(28, 28, 1)
    img = array_to_img(x_)
    img = img.resize(size=[64, 64])
    display(img)

In [ ]:
autoencoder.fit(X_train_noisy, X_train_noisy, epochs=1, validation_data=(X_test_noisy, X_test_noisy))

# Reconstruct Test Images

In [ ]:
X_encoded = encoder.predict(X_test_noisy)
X_reconstructed = decoder.predict(X_encoded)

noisy_test_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_test_noisy]
reconstructed_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_reconstructed]

# Visualize Reconstructed Images

In [ ]:
idxs = np.random.choice(len(X_test_noisy), size=5)
for idx in idxs:
    real_img, reconstructed_img = noisy_test_imgs[idx], reconstructed_imgs[idx]
    display(real_img); display(reconstructed_img)